# parte 1->
# 1 Definição do problema
# 2 carregamento dos dados
# 3 Seleção das variaveis

# 1 predição do consumo de energia levando em conta a temperatura e umidade que estar sendo medida através de sensores (iot)
Sobre o conjunto de dados
Modelagem-Preditiva-IoT-Previsão-de-Uso-de-Energia

Este projeto de Internet das Coisas (IoT) tem como principal propósito a criação de modelos preditivos para antecipar o consumo de energia de diversos aparelhos elétricos. Para esse fim, serão utilizados dados provenientes de múltiplas fontes, incluindo medições de sensores que monitoram temperatura e umidade em uma rede sem fio, informações de previsão meteorológica provenientes de uma estação localizada em um aeroporto, bem como dados sobre o consumo de energia por lâmpadas.

Nesse projeto de aprendizado de máquina, a etapa inicial consistirá em realizar uma filtragem dos dados, eliminando parâmetros que não possuem poder preditivo e selecionando os melhores recursos para a previsão.

Os dados foram coletados ao longo de um período de aproximadamente cinco meses, com medições realizadas a cada dez minutos. Para monitorar as condições de temperatura e umidade na residência, foi utilizada uma rede de sensores sem fio baseada na tecnologia ZigBee. Cada nó sem fio transmitia informações sobre a temperatura e umidade a cada três minutos, e posteriormente, uma média desses dados foi calculada em intervalos de dez minutos. Paralelamente, os registros de consumo de energia foram feitos a cada dez minutos através de medidores de energia específicos.

Os dados meteorológicos, obtidos da estação mais próxima ao aeroporto de Chievres, na Bélgica, foram adquiridos de uma fonte pública chamada Reliable Prognosis (rp5.ru) e integrados aos conjuntos de dados experimentais utilizando informações de data e hora.

# Com o intuito de avaliar os modelos de regressão e eliminar atributos que não se mostraram preditivos, foram incluídas duas variáveis investigativas no conjunto de dados.

O objetivo central deste projeto é desenvolver um modelo preditivo capaz de estimar o consumo de energia com base nos dados coletados através dos sensores IoT mencionados.

Os conjuntos de dados de treino e de teste estão em anexo.

# 2 carregamento dos dados

In [2]:

import pandas as pd
import sklearn as skl
import numpy as np
import matplotlib as plt


In [3]:
df_teste = pd.read_csv('dataset_testing.csv',sep = ',')
df_treino = pd.read_csv('dataset_training.csv',sep = ',') 

In [4]:
df = pd.concat([df_teste,df_treino])
df = df.sort_values(['date'], ascending=True)
df.head()


,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,NSM,WeekStatus,Day_of_week
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433,61200,Weekday,Monday
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195,61800,Weekday,Monday
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668,62400,Weekday,Monday
0,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389,63000,Weekday,Monday
3,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097,63600,Weekday,Monday


# 3 Seleção das variaveis

T > Temperatura

RH > Umidade Relativa (Relative Humidity)

T_out > Temperatura do lado de fora da casa

RH_out > Umidade do lado de fora da casa

mm_hg > Unidade de pressão 

In [6]:
df.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2', 'NSM', 'WeekStatus',
       'Day_of_week'],
      dtype='object')

In [13]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,NSM
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033,42907.129465
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634,24940.020831
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322,0.000000
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889,21600.000000
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653,43200.000000
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769,64200.000000
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530,85800.000000


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19735 entries, 0 to 14802
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  